In [ ]:
# %load ../source/convert_mnist.py
#! /usr/bin/python
"""
Converts MNIST data from original format to HDF5.
"""

import os.path as path

# To read the original MNIST data.
import gzip
import struct
import numpy as np

# To convert to HDF5.
import h5py

def read_mnist(image_fn, label_fn):
    file_names = [image_fn, label_fn]
    image_file, label_file = [gzip.open(fn, 'rb') for fn in file_names]

    def read_int(f):
        return struct.unpack('>I', f.read(4))[0]
    
    # Verify the contents of the headers.
    files            = [image_file, label_file]
    magic_1, magic_2 = [read_int(f) for f in files]
    count, count_2   = [read_int(f) for f in files]
    rows, cols       = [read_int(image_file) for _ in range(2)]

    assert magic_1 == 0x00000803
    assert magic_2 == 0x00000801
    assert count == count_2
    assert rows == 28 and cols == 28

    # Parse the data.
    labels = np.fromstring(label_file.read(count), dtype=np.uint8)
    images = np.fromstring(image_file.read(count * rows * cols), dtype=np.uint8)
    images.resize(count, rows, cols)
    return images, labels

def main():
    mnist_dir = "data/mnist"
    file_names = ["t10k-images-idx3-ubyte.gz", "t10k-labels-idx1-ubyte.gz",
        "train-images-idx3-ubyte.gz", "train-labels-idx1-ubyte.gz"]
    paths = [path.join(mnist_dir, fn) for fn in file_names]

    print("Parsing test images.")
    test_images, test_labels = read_mnist(paths[0], paths[1])
    print("Parsing train images.")
    train_images, train_labels = read_mnist(paths[2], paths[3])

    print("Creating HDF5 file.")
    output_file = h5py.File(path.join(mnist_dir, "mnist.hdf5"), 'w')
    train_group = output_file.create_group("/train")
    test_group = output_file.create_group("/test")
    train_group.create_dataset("inputs", data=train_images)
    train_group.create_dataset("targets", data=train_labels)
    test_group.create_dataset("inputs", data=test_images)
    test_group.create_dataset("targets", data=test_labels)

if __name__ == "__main__":
    main()


In [34]:
# Visually verify that we are parsing MNIST correctly.

import os.path as path
from PIL import Image

mnist_dir = "../data/mnist"
file_names = ["train-images-idx3-ubyte.gz", "train-labels-idx1-ubyte.gz"]
image_fn, label_fn = [path.join(mnist_dir, fn) for fn in file_names]

images, labels = read_mnist(image_fn, label_fn)

for i in range(5):
    print(labels[i])
    display(Image.fromarray(images[i], 'L'))

5


0


4


1


9
